In [1]:
import h5py
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import gdown
import shutil
import sys
import pandas as pd
import collections

from scipy.spatial.transform import Rotation

path = os.path.abspath("/home/jovyan/RobustPCR/Zero-overlap-test/SVDFormer")
sys.path.append(path)

import torch
from config_55 import cfg
from models.SVDFormer import Model
import utils.data_loaders
import utils.helpers
from utils.average_meter import AverageMeter
from utils.loss_utils import *
from models.model_utils import PCViews
from models.model_utils import fps_subsample
from tqdm import tqdm
import open3d as o3d

import torch.nn as nn
import torch.nn.functional as F
import torchgeometry as tgm
import open3d.ml.torch as ml3d

os.chdir("/home/jovyan/RobustPCR/Zero-overlap-test/gedi")
from backbones.pointnet2_ops_lib.pointnet2_ops.pointnet2_modules import PointnetSAModule

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# path = os.path.abspath("/home/jovyan/RobustPCR/Zero-overlap-test/gedi")
# sys.path.append(path)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.CONST.DEVICE

Jitting Chamfer 3D
Loaded JIT 3D CUDA chamfer distance
Loaded JIT 3D CUDA emd
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def plot_cloud_1(data1, name, idx, color = 'greens'):
    data1 = pd.DataFrame({'x1': data1[...,0],
                         'y1': data1[...,1],
                         'z1': data1[...,2],})
    fig = go.Figure(data=[
        go.Scatter3d(x=data1['x1'], y=data1['y1'], z=data1['z1'], 
                     mode='markers',
        marker=dict(
            size=2,
            color=data1['z1'],                # set color to an array/list of desired values
            colorscale=color,   # choose a colorscale
            opacity=0.8
        ))
    ])

    fig.write_html("/home/jovyan/RobustPCR/Zero-overlap-test/exp_out/" + idx + '_' + name +  ".html")

In [4]:
def plot_cloud_2(data1, data2, name, idx, color1 = 'reds', color2 = 'blues',):
    
    data1 = pd.DataFrame({'x1': data1[...,0],
                         'y1': data1[...,1],
                         'z1': data1[...,2],})

    data2 = pd.DataFrame({'x2': data2[...,0],
                          'y2': data2[...,1],
                          'z2': data2[...,2],})

    fig = go.Figure(data=[
        go.Scatter3d(x=data1['x1'], y=data1['y1'], z=data1['z1'], 
                     mode='markers',
        marker=dict(
            size=2,
            color=data1['z1'],                # set color to an array/list of desired values
            colorscale=color1,   # choose a colorscale
            opacity=0.2
        )),
        go.Scatter3d(x=data2['x2'], y=data2['y2'], z=data2['z2'],
                    mode='markers',
        marker=dict(
            size=2,
            color=data2['z2'],                # set color to an array/list of desired values
            colorscale=color2,   # choose a colorscale
            opacity=0.8
        ))
    ])

    fig.write_html("/home/jovyan/RobustPCR/Zero-overlap-test/exp_out/" + idx + '_' + name  + ".html")


In [5]:
def dump_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)


def process(subset):
    with open(f'modelnet40_ply_hdf5_2048/{subset}_files.txt') as f:
        lines = f.readlines()
    all_points = []
    all_normals = []
    all_labels = []
    for line in lines:
        filename = line.strip()
        print(filename.split("/")[-1])
        h5file = h5py.File(f'modelnet40_ply_hdf5_2048/{filename.split("/")[-1]}', 'r')
        all_points.append(h5file['data'][:])
        all_normals.append(h5file['normal'][:])
        all_labels.append(h5file['label'][:].flatten().astype(np.int))
    points = np.concatenate(all_points, axis=0)
    normals = np.concatenate(all_normals, axis=0)
    labels = np.concatenate(all_labels, axis=0)
    print(f'{subset} data loaded.')
    all_data = []
    num_data = points.shape[0]
    for i in range(num_data):
        all_data.append(dict(points=points[i], normals=normals[i], label=labels[i]))
    if subset == 'train':
        indices = np.random.permutation(num_data)
        num_train = int(num_data * 0.8)
        num_val = num_data - num_train
        train_indices = indices[:num_train]
        val_indices = indices[num_train:]
        train_data = [all_data[i] for i in train_indices.tolist()]
        dump_pickle(train_data, 'train.pkl')
        val_data = [all_data[i] for i in val_indices.tolist()]
        dump_pickle(val_data, 'val.pkl')
    else:
        dump_pickle(all_data, 'test.pkl')
        
def random_sample_plane():
    r"""Random sample a plane passing the origin and return its normal."""
    phi = np.random.uniform(0.0, 2 * np.pi)  # longitude
    theta = np.random.uniform(0.0, np.pi)  # latitude

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    normal = np.asarray([x, y, z])

    return normal

def random_crop_point_cloud_with_plane(points, p_normal=None, keep_ratio=0.7, normals=None):
    """Random crop a point cloud with a plane and keep num_samples points."""
    num_samples = int(np.floor(points.shape[0] * keep_ratio + 0.5))
    if p_normal is None:
        p_normal = random_sample_plane()  # (3,)
    distances = np.dot(points, p_normal)
    sel_indices = np.argsort(-distances)[:num_samples]  # select the largest K points
    remain_indices = [i for i in range(num_samples) if i not in sel_indices]
    points_1 = points[sel_indices]
    
    points_2 = points[remain_indices]

    if normals is not None:
        normals = normals[sel_indices]
        return points_1, points_2, normals
    else:
        return points_1, points_2
    
def use_o3d(pts, name, write_text=True):
    pcd = o3d.geometry.PointCloud()

    # the method Vector3dVector() will convert numpy array of shape (n, 3) to Open3D format.
    # see http://www.open3d.org/docs/release/python_api/open3d.utility.Vector3dVector_.html#open3d.utility.Vector3dVector
    #print('pts', pts.shape)
    pcd.points = o3d.utility.Vector3dVector(pts)

    # http://www.open3d.org/docs/release/python_api/open3d.io.write_point_cloud_.html#open3d.io.write_point_cloud
    o3d.io.write_point_cloud("%s.ply" % name, pcd, write_ascii=write_text)

    # read ply file
    # pcd = o3d.io.read_point_cloud('%s.ply' % name)


class tnet(nn.Module):

    def __init__(self,):
        super(tnet, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv1d(3, 256, 1, bias=False),
                                   nn.BatchNorm1d(256),
                                   nn.ReLU())

        self.conv2 = nn.Sequential(nn.Conv1d(256, 512, 1, bias=False),
                                   nn.BatchNorm1d(512),
                                   nn.ReLU())

        self.conv3 = nn.Sequential(nn.Conv1d(512, 1024, 1, bias=False),
                                   nn.BatchNorm1d(1024))

        self.fc1 = nn.Sequential(nn.Linear(1024, 512, bias=False),
                                 nn.BatchNorm1d(512),
                                 nn.ReLU())

        self.fc2 = nn.Sequential(nn.Linear(512, 256, bias=False),
                                 nn.BatchNorm1d(256),
                                 nn.ReLU())
        self._init_last_layer()

    def _init_last_layer(self):
        self.fc3 = nn.Linear(256, 9, bias=True)
        torch.nn.init.zeros_(self.fc3.bias)

    def _forward_last_layer(self, x):
        x = self.fc3(x)
        x = x + torch.eye(3, device='cuda').view(1, 9).repeat(x.size()[0], 1)
        x = x.view(-1, 3, 3)
        return x

    def forward(self, x):

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, x.shape[1])

        x = self.fc1(x)
        x = self.fc2(x)
        x = self._forward_last_layer(x)

        return x


class qnet(tnet):

    def _init_last_layer(self):
        self.fc3 = nn.Linear(256, 4, bias=True)
        torch.nn.init.zeros_(self.fc3.bias)

    def _forward_last_layer(self, x):
        quat = self.fc3(x)
        quat = quat + torch.tensor([1, 0, 0, 0], device='cuda').repeat(quat.size()[0], 1)
        quat = F.normalize(quat, p=2, dim=1)
        return quat


class PointNet2Feature(nn.Module):

    def __init__(self, dim=32):
        super(PointNet2Feature, self).__init__()

        self.use_xyz = True
        self.qnet = qnet()

        self.samodule1 = PointnetSAModule(
            npoint=128,
            radius=0.2,
            nsample=32,
            mlp=[3, 128, 128, 128],
            use_xyz=self.use_xyz,
        )

        self.samodule2 = PointnetSAModule(
            npoint=64,
            radius=0.4,
            nsample=16,
            mlp=[128+3, 256, 256, 256],
            use_xyz=self.use_xyz,
        )

        self.samodule3 = PointnetSAModule(
            mlp=[256+3, 512, 512, 1024], use_xyz=self.use_xyz
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(1024, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Dropout(0.3),
            nn.Linear(256, dim),
        )

    def _forward(self, pc):

        quat = self.qnet(pc)
        angle_axis = tgm.quaternion_to_angle_axis(quat)
        _trans = tgm.angle_axis_to_rotation_matrix(angle_axis)
        trans = _trans[:, :3, :3]
        pc = trans @ pc

        xyz = pc.transpose(1, 2).contiguous()
        xyz, features = self.samodule1(xyz, None)
        xyz, features = self.samodule2(xyz, features)
        xyz, features = self.samodule3(xyz, features)
        out = self.fc_layer(features.squeeze(-1))
        out = F.normalize(out, p=2, dim=1)

        return out, pc, trans

    def forward(self, xa, xp=torch.Tensor([])):
        if xp.nelement() == 0:
            f, _, _ = self._forward(xa)
            return f
        else:
            f0, pc0, trans0 = self._forward(xa)
            f1, pc1, trans1 = self._forward(xp)
            return f0, pc0, trans0, f1, pc1, trans1


class LRF(nn.Module):

    def __init__(self, patches_per_pair=256, samples_per_patch=256, eps=1e-12, r_lrf=1, device='cpu'):
        super(LRF, self).__init__()

        self.eps = eps
        self.r_lrf = r_lrf
        self.patches_per_pair = patches_per_pair
        self.samples_per_patch = samples_per_patch
        self.device = device

    def _forward(self, xp, xpi):

        B, N, c = xpi.size()
        xpi = xpi.contiguous()  # dim = B x 3 x N
        xp = xp.unsqueeze(2).contiguous()  # dim = B x 3 x 1

        # zp
        x = xp - xpi  # pi->p = p - pi
        xxt = torch.bmm(x, x.transpose(1, 2)) / c

        _, _, v = torch.svd(xxt.to(self.device))
        v = v.to(self.device)

        with torch.no_grad():
            sum_ = (v[..., -1].unsqueeze(1) @ x).sum(2)
            _sign = torch.ones((len(xpi), 1), device=self.device) - 2 * (sum_ < 0)

        zp = (_sign * v[..., -1]).unsqueeze(1)  # B x 1 x 3

        # xp
        x *= -1  # p->pi = pi - p
        norm = (zp @ x).transpose(1, 2)
        proj = norm * zp

        vi = x - proj.transpose(1, 2)

        x_l2 = torch.sqrt((x ** 2).sum(dim=1, keepdim=True))

        alpha = self.r_lrf - x_l2
        alpha = alpha * alpha
        beta = (norm * norm).transpose(1, 2)
        vi_c = (alpha * beta * vi).sum(2)

        xp = (vi_c / torch.sqrt((vi_c ** 2).sum(1, keepdim=True)))

        # yp
        yp = torch.cross(xp, zp.squeeze(), dim=1)

        lrf = torch.cat((xp.unsqueeze(2), yp.unsqueeze(2), zp.transpose(1, 2)), dim=2)

        return lrf

    def forward(self, x0, x0i, x1=None, x1i=None):

        # compute local reference frames
        lrf0 = self._forward(x0, x0i)
        inds = np.random.choice(x0i.shape[2], self.samples_per_patch, replace=False)
        _out_x0 = (x0i[..., inds] - x0.unsqueeze(-1)) / self.r_lrf
        out_x0 = lrf0.transpose(1, 2) @ _out_x0

        if x1 is None:
            return out_x0

        lrf1 = self._forward(x1, x1i)
        inds = np.random.choice(x1i.shape[2], self.samples_per_patch, replace=False)
        _out_x1 = (x1i[..., inds] - x1.unsqueeze(-1)) / self.r_lrf
        out_x1 = lrf1.transpose(1, 2) @ _out_x1

        return out_x0, out_x1


class GeDi:

    def __init__(self, config):
        self.dim = config['dim']
        self.samples_per_batch = config['samples_per_batch']
        self.samples_per_patch_lrf = config['samples_per_patch_lrf']
        self.samples_per_patch_out = config['samples_per_patch_out']
        self.r_lrf = config['r_lrf']

        self.lrf = LRF(patches_per_pair=self.samples_per_batch,
                       samples_per_patch=self.samples_per_patch_out,
                       r_lrf=self.r_lrf,
                       device='cpu')

        self.gedi_net = PointNet2Feature(dim=self.dim)
        self.gedi_net.load_state_dict(torch.load(config['fchkpt_gedi_net'])['pnet_model_state_dict'])
        self.gedi_net.cuda().eval()

    def compute(self, pts, pcd):

        radii = self.r_lrf * torch.ones((len(pts)))

        out = ml3d.ops.radius_search(pcd, pts, radii,
                                     points_row_splits=torch.LongTensor([0, len(pcd)]),
                                     queries_row_splits=torch.LongTensor([0, len(pts)]))

        pcd_desc = np.empty((len(pts), self.dim))

        for b in range(int(np.ceil(len(pts) / self.samples_per_batch))):

            i_start = b * self.samples_per_batch
            i_end = (b + 1) * self.samples_per_batch
            if i_end > len(pts):
                i_end = len(pts)

            x = np.empty((i_end - i_start, 3, self.samples_per_patch_lrf))

            j = 0
            for i in range(i_start, i_end):

                _inds = out[0][out[1][i]:out[1][i + 1]]
                try:
                    inds = np.random.choice(_inds.numpy(), size=self.samples_per_patch_lrf, replace=False)
                except:
                    # print('[w] got patch with few points -> {}. Padding with replicas ...'.format(len(pt_nn)))
                    inds = np.r_[_inds, np.random.choice(_inds.numpy(), self.samples_per_patch_lrf - len(_inds))]

                x[j] = pcd[inds].T

                j += 1

            x = torch.Tensor(x)

            patch = self.lrf(pts[i_start:i_end], x)

            with torch.no_grad():
                f = self.gedi_net(patch.cuda())

            pcd_desc[i_start:i_end] = f.cpu().detach().numpy()[:i_end - i_start]

        return pcd_desc

In [10]:
def test(xyz, data, idx):

    os.chdir("/home/jovyan/RobustPCR/Zero-overlap-test/gedi")
    use_o3d(xyz, 0)
    use_o3d(data, 2)
    #print('xyz', xyz, 'data', data)

    config = {'dim': 32,                                            # descriptor output dimension
              'samples_per_batch': 500,                             # batches to process the data on GPU
              'samples_per_patch_lrf': 4000,                        # num. of point to process with LRF
              'samples_per_patch_out': 512,                         # num. of points to sample for pointnet++
              'r_lrf': .5,                                          # LRF radius
              'fchkpt_gedi_net': 'data/chkpts/3dmatch/chkpt.tar'}   # path to checkpoint

    voxel_size = .01
    patches_per_pair = 5000

    # initialising class
    gedi = GeDi(config=config)

    # getting a pair of point clouds
    pcd0 = o3d.io.read_point_cloud('/home/jovyan/RobustPCR/Zero-overlap-test/gedi/0.ply')
    pcd1 = o3d.io.read_point_cloud('/home/jovyan/RobustPCR/Zero-overlap-test/gedi/2.ply')

    pcd0.paint_uniform_color([1, 0.706, 0])
    pcd1.paint_uniform_color([0, 0.651, 0.929])

    # estimating normals (only for visualisation)
    pcd0.estimate_normals()
    pcd1.estimate_normals()

    o3d.visualization.draw_geometries([pcd0, pcd1])

    # randomly sampling some points from the point cloud
    inds0 = np.random.choice(np.asarray(pcd0.points).shape[0], patches_per_pair, replace=True)
    inds1 = np.random.choice(np.asarray(pcd1.points).shape[0], patches_per_pair, replace=True)

    pts0 = torch.tensor(np.asarray(pcd0.points)[inds0]).float()
    pts1 = torch.tensor(np.asarray(pcd1.points)[inds1]).float()

    # applying voxelisation to the point cloud
    pcd0 = pcd0.voxel_down_sample(voxel_size)
    pcd1 = pcd1.voxel_down_sample(voxel_size)

    _pcd0 = torch.tensor(np.asarray(pcd0.points)).float()
    _pcd1 = torch.tensor(np.asarray(pcd1.points)).float()

    # computing descriptors
    pcd0_desc = gedi.compute(pts=pts0, pcd=_pcd0)
    pcd1_desc = gedi.compute(pts=pts1, pcd=_pcd1)

    # preparing format for open3d ransac
    pcd0_dsdv = o3d.pipelines.registration.Feature()
    pcd1_dsdv = o3d.pipelines.registration.Feature()

    pcd0_dsdv.data = pcd0_desc.T
    pcd1_dsdv.data = pcd1_desc.T

    _pcd0 = o3d.geometry.PointCloud()
    _pcd0.points = o3d.utility.Vector3dVector(pts0)
    _pcd1 = o3d.geometry.PointCloud()
    _pcd1.points = o3d.utility.Vector3dVector(pts1)

    # applying ransac
    est_result01 = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        _pcd0,
        _pcd1,
        pcd0_dsdv,
        pcd1_dsdv,
        mutual_filter=True,
        max_correspondence_distance=.02,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=3,
        checkers=[o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(.9),
                  o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(.02)],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(50000, 1000))

    # applying estimated transformation
    pcd0.transform(est_result01.transformation)

    print(est_result01)

    arr0 = np.asarray(pcd0.points)
    arr1 = np.asarray(pcd1.points)
    
    
    plot_cloud_2(arr0, arr1 , 'gedi_', idx)
    

    data1 = pd.DataFrame({'x1': arr0[...,0],
                         'y1': arr0[...,1],
                         'z1': arr0[...,2],})

    data2 = pd.DataFrame({'x2': arr1[...,0],
                          'y2': arr1[...,1],
                          'z2': arr1[...,2],})

    fig = go.Figure(data=[
        go.Scatter3d(x=data1['x1'], y=data1['y1'], z=data1['z1'], 
                     mode='markers',
        marker=dict(
            size=2,
            color=data1['z1'],                # set color to an array/list of desired values
            colorscale='reds',   # choose a colorscale
            opacity=0.2
        )),
        go.Scatter3d(x=data2['x2'], y=data2['y2'], z=data2['z2'],
                    mode='markers',
        marker=dict(
            size=2,
            color=data2['z2'],                # set color to an array/list of desired values
            colorscale='blues',   # choose a colorscale
            opacity=0.8
        ))
    ])

    fig.write_html("/home/jovyan/RobustPCR/Zero-overlap-test/exp_out/" + idx + "_test1.html")

    fig = make_subplots(rows=1, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}]],
                    )

    fig.add_trace(go.Scatter3d(x=data1['x1'], y=data1['y1'], z=data1['z1'], 
                     mode='markers',
        marker=dict(
            size=2,
            color=data1['z1'],                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.2
        )), 1, 1)
    fig.add_trace(go.Scatter3d(x=data2['x2'], y=data2['y2'], z=data2['z2'],
                    mode='markers',
        marker=dict(
            size=2,
            color=data2['z2'],                # set color to an array/list of desired values
            colorscale='thermal',   # choose a colorscale
            opacity=0.8
        )), 1, 2)
    fig.update_layout()

    ("/home/jovyan/RobustPCR/Zero-overlap-test/exp_out/" + idx + "_test2.html")
    
    
    return est_result01

SVDFormer

In [ ]:
os.chdir("/home/jovyan/RobustPCR/Zero-overlap-test/SVDFormer")

dataset_loader = utils.data_loaders.DATASET_LOADER_MAPPING[cfg.DATASET.TEST_DATASET](cfg)
test_data_loader = torch.utils.data.DataLoader(dataset=dataset_loader.get_dataset(
utils.data_loaders.DatasetSubset.TEST),
                                          batch_size=1,
                                          num_workers=4,
                                          collate_fn=utils.data_loaders.collate_fn_55,
                                          pin_memory=True,
                                          shuffle=False)

# Setup networks and initialize networks
model = Model(cfg)
if torch.cuda.is_available():
    model = torch.nn.DataParallel(model).cuda()

checkpoint = torch.load(cfg.CONST.WEIGHTS)
model.load_state_dict(checkpoint['model'])

# Switch models to evaluation mode
model.eval()

n_samples = len(test_data_loader)
test_losses = AverageMeter(['CD', 'DCD', 'F1'])
test_metrics = AverageMeter(['CD','DCD','F1'])
category_metrics = dict()
mclass_metrics = AverageMeter(['CD','DCD','F1'])
render = PCViews(TRANS=-cfg.NETWORK.view_distance, RESOLUTION=224)

# Eval settings
crop_ratio = {
    'easy': 1 / 5,
    'median': 1 / 2,
    'hard': 3 / 4
}
choice = [torch.Tensor([1, 1, 1]), torch.Tensor([1, 1, -1]), torch.Tensor([1, -1, 1]), torch.Tensor([-1, 1, 1]),
          torch.Tensor([-1, -1, 1]), torch.Tensor([-1, 1, -1]), torch.Tensor([1, -1, -1]),
          torch.Tensor([-1, -1, -1])]

mode = cfg.CONST.mode

print('Start evaluating (mode: {:s}) ...'.format(mode))
#_R, _T = list(), list()
epoch_idx = -1
test_writer=None
metrics = collections.defaultdict(dict)
# Testing loop
with tqdm(test_data_loader) as t:
    for batch_idx, (taxonomy_id, model_ids, data) in enumerate(t):
        #print(model_ids, taxonomy_id, model_ids, data['gtcloud'].shape)
        
        R = Rotation.from_euler('z', [30], degrees=True).as_matrix()[0]
        T = np.array([[0.1],[0.1],[0.1]]).T
        
        if batch_idx <1000:
            taxonomy_id = taxonomy_id[0] if isinstance(taxonomy_id[0], str) else taxonomy_id[0].item()
            with torch.no_grad():
                for k, v in data.items():
                    data[k] = utils.helpers.var_or_cuda(v)
                
                sample = data['gtcloud'] 
                #print(sample.shape)
                plot_cloud_1(sample.cpu().numpy()[0], 'sample_', str(model_ids[0]))
                
                init_1, init_2 = random_crop_point_cloud_with_plane(sample.cpu().numpy()[0], keep_ratio=0.6)
                
                #### нужна вторая часть облака 
                
                gt = torch.Tensor(init_1).to('cuda') @ torch.Tensor(R).to('cuda') + torch.Tensor(T).to('cuda')
                gt = torch.unsqueeze(gt, 0)
                _, npoints, _ = gt.size()
                plot_cloud_2(init_1, init_2, 'points_', str(model_ids[0]))
                plot_cloud_2(gt.cpu().numpy()[0], init_2, 'rotation_', str(model_ids[0]))
                
                # partial clouds from fixed viewpoints
                #num_crop = int(npoints * crop_ratio[mode])
                #for partial_id, item in enumerate(choice):
                #    partial, _ = utils.helpers.seprate_point_cloud(gt, npoints, num_crop, fixed_points = item)
                #    _partial = partial
                    
                partial = fps_subsample(gt, 2048)
                #partial = partial #@ torch.Tensor(R).to('cuda') #+ torch.Tensor(T).to('cuda')
                partial_depth = torch.unsqueeze(render.get_img(partial), 1)
                pcds_pred = model(partial.contiguous(),partial_depth)
                cdl1,cdl2,f1 = calc_cd(pcds_pred[-1],gt,calc_f1=True)
                dcd,_,_ = calc_dcd(pcds_pred[-1],gt)

                cd = cdl2.mean().item() * 1e3
                dcd = dcd.mean().item()
                f1 = f1.mean().item()

                _metrics = [cd, dcd, f1]
                test_losses.update([cd, dcd, f1])

                test_metrics.update(_metrics)

                t.set_description('Test[%d/%d]  Losses = %s Metrics = %s' %(batch_idx, n_samples,  ['%.4f' % l for l in test_losses.avg()
                                                                                ], ['%.4f' % m for m in _metrics]))
                partial_depth = partial_depth.cpu().numpy()
                #_partial = _partial.cpu().numpy()
                partial = partial.cpu().numpy()
                final = pcds_pred[-1].cpu().numpy()
                #init = data['gtcloud'].cpu().numpy()[0]
                #final = gt
                
                plot_cloud_2(final[0], init_2, 'svdf_', str(model_ids[0]))
                plot_cloud_2(final[0], partial[0], 'svdf_partial', str(model_ids[0]))
                val = test(final[0], init_2, str(model_ids[0]))
               
        else:
            break
        _R = val.transformation[:3,:3]
        _T = val.transformation[:3,3]
        metrics[str(model_ids[0])]['R'] = R
        metrics[str(model_ids[0])]['T'] = T
        metrics[str(model_ids[0])]['_R'] = _R
        metrics[str(model_ids[0])]['_T'] = _T
        metrics[str(model_ids[0])]['val'] = val
        metrics[str(model_ids[0])]['_T'] = _T

        #metrics[str(model_ids[0])]['rotation_error'] = rotation_error(R, _R)
        #metrics[str(model_ids[0])]['translation_error'] = translation_error(T, _T)
        metrics[str(model_ids[0])]['Error_R'] = Error_R(R, _R)
        metrics[str(model_ids[0])]['Error_t'] = Error_t(T, _T)
                   
                    
    print('============================ TEST RESULTS ============================')
    print('Taxonomy', end='\t')
    print('#Sample', end='\t')
    for metric in test_metrics.items:
        print(metric, end='\t')
    print()
    print(test_metrics.avg())
    print('Overall', end='\t\t\t')
    for value in test_metrics.avg():
        print(test_metrics.avg())
        print('%.4f' % value, end='\t')
    print('\n')

    print('Epoch ', epoch_idx, end='\t')
    for value in test_losses.avg():
        print('%.4f' % value, end='\t')
    print('\n')

    # Add testing results to TensorBoard
    if test_writer is not None:
        test_writer.add_scalar('Loss/Epoch/cd', test_losses.avg(0), epoch_idx)
        test_writer.add_scalar('Loss/Epoch/dcd', test_losses.avg(1), epoch_idx)
        test_writer.add_scalar('Loss/Epoch/f1', test_losses.avg(2), epoch_idx)
        for i, metric in enumerate(test_metrics.items):
            test_writer.add_scalar('Metric/%s' % metric, test_metrics.avg(i), epoch_idx)

                    # sys.exit()
    

Complete collecting files of the dataset. Total files: 10518
Start evaluating (mode: easy) ...


Test[0/10518]  Losses = ['8.5565', '0.4734', '0.8429'] Metrics = ['8.5565', '0.4734', '0.8429']:   0%|          | 0/10518 [00:02<?, ?it/s]

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
RegistrationResult with fitness=2.854000e-01, inlier_rmse=1.147495e-02, and correspondence_set size of 1427
Access transformation to get result.


Test[1/10518]  Losses = ['9.5995', '0.4728', '0.8633'] Metrics = ['10.6424', '0.4722', '0.8838']:   0%|          | 1/10518 [00:13<38:42:23, 13.25s/it]

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
RegistrationResult with fitness=7.760000e-02, inlier_rmse=1.269448e-02, and correspondence_set size of 388
Access transformation to get result.


Test[2/10518]  Losses = ['9.4655', '0.4879', '0.6965'] Metrics = ['9.1977', '0.5182', '0.3629']:   0%|          | 2/10518 [00:26<38:09:49, 13.06s/it] 

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
metrics

In [11]:
def Error_R(r1, r2):
    '''
    Calculate isotropic rotation degree error between r1 and r2.
    :param r1: shape=(B, 3, 3), pred
    :param r2: shape=(B, 3, 3), gt
    :return:
    '''
    r2_inv = np.transpose(r2)#.transpose(0, 2, 1)
    r1r2 = np.matmul(r2_inv, r1)
    tr = r1r2[0, 0] + r1r2[1, 1] # + r1r2[:, 2, 2]
    rads = np.arccos(np.clip((tr - 1) / 2, -1, 1))
    degrees = rads / np.pi * 180
    return degrees

In [12]:
def Error_t(t1, t2):
    '''
    calculate translation mse error.
    :param t1: shape=(B, 3)
    :param t2: shape=(B, 3)
    :return:
    '''
    #assert t1.shape == t2.shape
    error_t = np.sqrt(np.sum((t1 - t2) ** 2, axis=1))
    return error_t

In [13]:
def rotation_error(R1, R2):
    """
    Torch batch implementation of the rotation error between the estimated and the ground truth rotatiom matrix. 
    Rotation error is defined as r_e = \arccos(\frac{Trace(\mathbf{R}_{ij}^{T}\mathbf{R}_{ij}^{\mathrm{GT}) - 1}{2})

    Args: 
        R1 (torch tensor): Estimated rotation matrices [b,3,3]
        R2 (torch tensor): Ground truth rotation matrices [b,3,3]

    Returns:
        ae (torch tensor): Rotation error in angular degreees [b,1]

    """
    print(R1.shape, R2.shape)
    R1 = torch.from_numpy(R1).to('cuda')
    R2 = torch.from_numpy(R2).to('cuda')
    print('2', R1, R2)
    R_ = torch.matmul(R1.transpose(0,1), R2)
    e = torch.stack([(torch.trace(R_[_, :, :]) - 1) / 2 for _ in range(R_.shape[0])], dim=0).unsqueeze(1)

    # Clamp the errors to the valid range (otherwise torch.acos() is nan)
    e = torch.clamp(e, -1, 1, out=None)

    ae = torch.acos(e)
    pi = torch.Tensor([np.pi])
    ae = 180. * ae / pi.to(ae.device).type(ae.dtype)

    return ae


def translation_error(t1, t2):
    """
    Torch batch implementation of the rotation error between the estimated and the ground truth rotatiom matrix. 
    Rotation error is defined as r_e = \arccos(\frac{Trace(\mathbf{R}_{ij}^{T}\mathbf{R}_{ij}^{\mathrm{GT}) - 1}{2})

    Args: 
        t1 (torch tensor): Estimated translation vectors [b,3,1]
        t2 (torch tensor): Ground truth translation vectors [b,3,1]

    Returns:
        te (torch tensor): translation error in meters [b,1]

    """
    t1 = torch.from_numpy(t1).to('cuda')
    t2 = torch.from_numpy(t2).to('cuda')
    
    return torch.norm(t1-t2, dim=(1, 2))